<a href="https://colab.research.google.com/github/zum-m/01_jouhougakugairon/blob/quantium/Copy_of_%E6%BC%94%E7%BF%92%E7%AC%AC11%E5%9B%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **11.グローバーのアルゴリズム**

グローバーのアルゴリズムは量子コンピュータを利用して、データベース検索を高速におこなうためのアルゴリズムです。



##**11.1 演習のための準備**
以下のコードを実行し、qiskitをインストールしましょう。

In [ ]:
!pip install qiskit==0.39.4

さらに以下のコードを実行して、Qiskitによる学習を支援するライブラリーqiskit-textbookと量子回路を描画するためのライブラリーpylatexencもインストールします。

In [ ]:
!pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

In [ ]:
!pip install pylatexenc

演習で使用するパッケージをqiskitライブラリーからimportします。  
以下のコードを実行しましょう。  


In [ ]:
# initialization
import matplotlib.pyplot as plt
import numpy as np
import math

from numpy import pi
from numpy.random import randint
from math import gcd
from tabulate import tabulate
from fractions import Fraction
# importing Qiskit
from qiskit import *
from qiskit import Aer
from qiskit import BasicAer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit,execute, ClassicalRegister, QuantumRegister, assemble, transpile
from qiskit_textbook.widgets import dj_widget
from qiskit_textbook.tools import array_to_latex
from qiskit.tools.monitor import job_monitor

# import basic plot tools
from qiskit.visualization import plot_histogram, array_to_latex
from qiskit.visualization import plot_bloch_multivector



##**11.2 アルゴリズムの手順**


$n$ビット列$\{x_1 x_2 \cdots x_n \}$で表現される$N=2^n$個のデータ集合があるとします。データは未整序（順序がバラバラ）に置かれています。この中から特定のデータ$\omega$を取り出す、つまり$\omega$を検索することを考えます。

無作為にデータを取り出し、それがデータ$\omega$であるかをチェックしていくのが古典的検索方法です。この方法では平均で$N/2$回、最悪で$N$回のチェックが必要です。

グローバーのアルゴリズムでは、量子コンピューターが実現する「振幅増幅」と呼ばれるテク二ックが利用されます。これによりおおよそ$\sqrt{N}$回のステップでデータ$\omega$を探し出すことができます。

* **手順１**

初期状態$|00\cdots 0\rangle = |0\rangle ^{⊗n}$にアダマールゲートを適用して重ね合わせの状態をつくります。
$$H^{⊗n}|0⟩^{⊗n}=\frac{1}{\sqrt{2^n}}\sum_{x=0}^{2^n-1}|x⟩=|s⟩$$
$x$は$2^{n-1}x_1+2^{n-2}x_2+\cdots +x_n$を意味します。  
検索したいデータ$\omega$に対応したビット列$\{\omega_1 \omega_2 \cdots \omega_n\}$は$\omega=2^{n-1}\omega_1+2^{n-2}\omega_2+\cdots +\omega_n$
を満たします。  
この状態に観測をおこなえば、$|\omega \rangle $が得られる確率は$1/2^n$です。


* **手順２**

重ね合わせの状態$|s⟩$に以下の式で定義される量子オラクルゲート$U_f$を適用します．
$$
U_f|x\rangle=\left\{\begin{aligned}
|x\rangle & \text { if } x \neq \omega \\
-|x\rangle & \text { if } x=\omega
\end{aligned}\right.
$$
$$U_f|s⟩=\frac{1}{\sqrt{2^n}}(\sum_{x=0,x\neq \omega}^{2^n-1}U_f|x⟩+U_f|\omega ⟩)\\
=\frac{1}{\sqrt{2^n}}(\sum_{x=0,x\neq \omega}^{2^n-1}|x⟩-|\omega ⟩)\\
=\frac{1}{\sqrt{2^n}}(\sum_{x=0}^{2^n-1}|x⟩-2|\omega ⟩)\\
=|s⟩-\frac{2}{\sqrt{2^n}}|\omega ⟩$$

* **手順３**
次に$U_s=2|s⟩⟨s|-I$と定義されるユニタリーゲートを適用します．
このゲートはひとつのビット列状態$|x⟩$を
$$U_s|x⟩=2|s⟩⟨s|x⟩-|x⟩=\frac{2}{\sqrt{2^n}}|s⟩-|x⟩$$
に変換しますが、ビット列の重ね合わせ$|s⟩$に対しては不変となります。
$$U_s|s⟩=2|s⟩⟨s|s⟩-|s⟩=2|s⟩-|s⟩=|s⟩$$
したがって、
$$U_sU_f|s⟩=U_s|s⟩-\frac{2}{\sqrt{2^n}}U_s|\omega ⟩\\
=|s⟩-\frac{2}{\sqrt{2^n}}(\frac{2}{\sqrt{2^n}}|s⟩-|\omega⟩)\\
=\frac{2^n-4}{2^n}|s⟩+\frac{2}{\sqrt{2^n}}|\omega⟩$$
となります。上式はさらに
$$\frac{2^n-4}{2^n\sqrt{2^n}}\sum_{x=0,x\neq \omega}^{2^n-1}|x⟩+(\frac{2^n-4}{2^n\sqrt{2^n}}+\frac{2}{\sqrt{2^n}})|\omega⟩\\
=\frac{2^n-4}{2^n\sqrt{2^n}}\sum_{x=0,x\neq \omega}^{2^n-1}|x⟩+(3-\frac{4}{2^n})\frac{1}{\sqrt{2^n}}|\omega⟩$$
と書けることに注意しましょう。

つまりこの時点で観測をおこなえば、
$$(3-\frac{4}{2^n})^2\frac{1}{{2^n}}$$
の確率で$|\omega ⟩$を得ることができます。  
手順１の時点での確率と比較するとおおよそ9倍に上昇しています。

$U_f,U_s$ゲートを利用したこの操作を「振幅増幅」と呼びます。

* **手順４**

振幅増幅を複数回おこなうことで、$|\omega ⟩$を得る確率を１に近づけていきます。


##**11.3 振幅増幅の最適回数**

グローバーのアルゴリズムでは、  
$|\omega⟩$を観測する確率を最大にする
振幅増幅の最適な回数が問題となります．  
ここでは重ね合わせの状態
$$|s⟩=\frac{1}{\sqrt{2^n}}\sum_{x=0,x\neq \omega}^{2^n-1}|x⟩+\frac{1}{\sqrt{2^n}}|\omega ⟩$$
を$\sin θ=\frac{1}{\sqrt{2^n}}$と置いて
$$|s⟩=\cos θ|\omega^\perp⟩+\sin θ|\omega ⟩$$
と書き直すことでこの問題を議論します。  


量子オラクルゲート$U_f$により、状態$|s⟩$は
$$U_f|s⟩=\cos θ|\omega^\perp⟩-\sin θ|\omega ⟩$$
に変換され、さらにユニタリーゲート$U_s=2|s⟩⟨s|-I$により、
$$U_sU_f|s⟩=\cos θ\left(2|s⟩⟨s|\omega^\perp⟩-|\omega^\perp ⟩\right)-\sin θ\left(2|s⟩⟨s|\omega⟩-|\omega ⟩\right)$$
に変換されます。$⟨s|\omega^\perp⟩=\cos θ\ ,\ ⟨s|\omega⟩=\sin θ $ より、  
上式は
$$U_sU_f|s⟩=\left(2\cos^2 θ-2\sin^2 θ \right)|s⟩-\cos θ|\omega^\perp⟩+\sin θ |\omega⟩\\
=\left(4\cos^3 θ-3\cos θ\right)|\omega^\perp⟩+\left(-4\sin^3 θ+3\sin θ\right)|\omega⟩\\
=\cos 3θ|\omega^\perp⟩+\sin 3\theta|\omega⟩$$
とまとめられます。（最後の等式では３倍角の公式を用いました。）


一般に振幅増幅の作用$U_sU_f$を$j$回施すと、状態は
$$\left(U_sU_f\right)^j|s⟩=\cos (2j+1)\theta |\omega^\perp⟩+\sin (2j+1)\theta|\omega⟩$$
となります。  
したがって、振幅増幅の最適な回数は$(2j+1)\theta$が最も$\pi/2$に近い整数$j$
です。$\sin θ=\frac{1}{\sqrt{2^n}}$であることに注意します。
$$\theta=\arcsin(\frac{1}{\sqrt{2^n}})$$
最適回数$j$は
$$j≈\frac{\pi}{4}\frac{1}{\arcsin(\frac{1}{\sqrt{2^n}})}-\frac{1}{2}≈\frac{\pi \sqrt{N}}{4}$$
を満たします。

例えば、$n=2$の場合、$\arcsin(1/2)=\frac{\pi}{6}$となるので、  
最適回数は$j=1$です。



##**11.4 量子回路の作成とシミュレーション**



## **11.4.1 $n=2$の場合**

ここでは2量子ビットのグローバーのアルゴリズムを実装した量子回路を
作成します。ビット列$\{00,01,10,11\}$から特定のビット列$11$を探索することを想定します。

まず2量子ビットの量子回路を準備します。
```
n = 2
grover_circuit = QuantumCircuit(n)
```

* **重ね合わせ状態の作成**

「手順１：ビット列の重ね合わせの状態を作成する」関数を定義しましょう。
```
def initialize_s(qc, qubits):
    """qcの 'qubits' にH-gate を適用"""
    for q in qubits:
        qc.h(q)
    return qc
```

この関数を「grover_circuit」に適用します。
```
grover_circuit = initialize_s(grover_circuit, [0,1])
grover_circuit.draw('mpl')
```

* **量子オラクル$U_f$ゲートの実装**

次に$|\omega ⟩=|11⟩$に対応した量子オラクルゲート$U_f$を実行する回路を作成します。これは制御Zゲートの適用に対応します。  
量子ビット$q_0$が制御ビット、$q_1$が標的ビットとなります。
つまり、$q_0$の状態が$1$のときに、$q_1$の状態に
$$
Z=\left[\begin{array}{cccccccc}
1 & 0  \\
0 & -1  \\
\end{array}\right]
$$
が演算されます。  
結果、状態$|11⟩$の振幅にだけマイナスがかかり、$U_f$の目的が果たされます。
簡単ですね。
```
grover_circuit.cz(0,1) # オラクル
grover_circuit.draw('mpl')
```

* **ユニタリーゲート$U_s$の実装**

ユニタリーゲート$U_s=2|s⟩⟨s|-I$は以下のようにして実装できます。

まず、制御を$q_0$,標的を$q_1$とする制御Zゲートを考えます。  
これに相当する行列は
$$CZ=|00⟩⟨00|+|01⟩⟨01|+|10⟩⟨10|-|11⟩⟨11|$$
と書けます。

次に制御Zゲートを$X^{⊗2}$でパックします。  
$$X^{⊗2}CZX^{⊗2}=X^{⊗2}|00⟩⟨00|X^{⊗2}+X^{⊗2}|01⟩⟨01|X^{⊗2}+X^{⊗2}|10⟩⟨10|X^{⊗2}-X^{⊗2}|11⟩⟨11|X^{⊗2}\\
=|11⟩⟨11|+|10⟩⟨10|+|01⟩⟨01|-|00⟩⟨00|$$
この行列は
$$X^{⊗2}CZX^{⊗2}=I-2|00⟩⟨00|$$
と書けることに注意します。
$I$は恒等演算(単位行列)なので
$$I=|00⟩⟨00|+|01⟩⟨01|+|10⟩⟨10|+|11⟩⟨11|$$
ですね。

さらに$X^{⊗2}CZX^{⊗2}$をアダマールゲート$H$でパックします。  
$$H^{\otimes 2}X^{⊗2}CZX^{⊗2} H^{\otimes 2}=I-2H^{\otimes 2}|00⟩⟨00|H^{\otimes 2}=I-2|s⟩⟨s|$$
となり、$-U_s$と一致します。  
全体にマイナスがかかりますが、計算の本質は変わりません。

```
grover_circuit.h([0,1])
grover_circuit.x([0,1])
grover_circuit.cz(0,1)
grover_circuit.x([0,1])
grover_circuit.h([0,1])
grover_circuit.draw('mpl')
```

これで一回目の振幅増幅が実行できます。  
ただし$n=2$の場合は1回で十分です。
前節で説明したように、理論上、$|\omega ⟩=|00⟩$を観測する確率は１になるはずです。  
シミュレーションで確認してみましょう。
```
grover_circuit.measure_all()

aer_sim = Aer.get_backend('aer_simulator')
qobj = assemble(grover_circuit)
result = aer_sim.run(qobj).result()
counts = result.get_counts()
plot_histogram(counts)
```

## **11.4.2 $n=3$の場合**

3量子ビットのグローバーのアルゴリズムを実装した量子回路を
作成します。  
ただし、応用としてビット列$\{000,001,\cdots, 111\}$から2つの特定のビット列$101, 110$を探索することを想定します。

3量子ビットの量子回路を準備しましょう。
```
n = 3
grover_circuit2 = QuantumCircuit(n)

```




* **重ね合わせ状態の作成**
```
grover_circuit2.clear()
grover_circuit2 = initialize_s(grover_circuit2, [0,1,2])
grover_circuit2.draw('mpl')
```

* **量子オラクル$U_f$ゲートの実装**

$|101⟩$と$|110\rangle$の振幅にマイナスをかける量子オラクルゲート$U_f$を
適用します。  
ふたつの制御Zゲートを使用します。  
ひとつについては$q_2$が制御ビット、$q_1$が標的ビットとなり、
もうひとつについては$q_2$が制御ビット、$q_0$が標的ビットとなります。

```
grover_circuit2.cz(2,1)
grover_circuit2.cz(2,0)
grover_circuit2.draw('mpl')
```

* **ユニタリーゲート$U_s$の実装**

前節と同様の議論から、一般的に
$$H^{\otimes n}X^{⊗n}C^{n-1}ZX^{⊗n} H^{\otimes n}$$
は$-U_s$と一致します。  
$C^{n-1}Z$はマルチ制御Zゲートを意味し、制御ビットがn-1個あります。
ここではn量子ビット用の$U_s$を作成する関数を一般的に定義しましょう。

```
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # 全量子ビットにHゲートを適用
    for qubit in range(nqubits):
        qc.h(qubit)
    # 全量子ビットにXゲートを適用
    for qubit in range(nqubits):
        qc.x(qubit)
    # マルチ制御Zゲートを適用
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # マルチ制御トフォリ
    qc.h(nqubits-1)
    # 全量子ビットにXゲートを適用
    for qubit in range(nqubits):
        qc.x(qubit)
    # 全量子ビットにHゲートを適用
    for qubit in range(nqubits):
        qc.h(qubit)
    # Diffuserをゲートにします
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s
```

上に定義した$U_s$を回路に適用しましょう。
```
grover_circuit2.append(diffuser(3), [0,1,2])
grover_circuit2.measure_all()
grover_circuit2.draw('mpl')
```

これで完成です。
シミュレーションをおこなってみましょう。
```
aer_sim = Aer.get_backend('aer_simulator')
transpiled_grover_circuit2 = transpile(grover_circuit2, aer_sim)
qobj = assemble(transpiled_grover_circuit2)
results = aer_sim.run(qobj).result()
counts = results.get_counts()
plot_histogram(counts)
```

## **演習課題11**

###問題

以下のコードを実行してください。

In [ ]:
from qiskit_textbook.problems import grover_problem_oracle
n = 4
oracle = grover_problem_oracle(n, variant=2)  # n量子ビットのオラクルの0番目の変数
qc = QuantumCircuit(n)
qc.append(oracle, [0,1,2,3])



qc.draw('mpl')

qiskit_textbookにはグローバーのアルゴリズム用のオラクルゲート$U_f$が用意されています。  
このOracle(n=4,var=2)は4量子ビット用で、$0000$から$1111$までの16あるデータのうち、あるひとつのデータを探索するためのものです。（そのデータは秘密にされています。）

Oracle(n=4,var=2)ゲートを適用したグローバーのアルゴリズムの量子回路を作成し、  
シミュレーションをおこなってください。
ただし、探索対象のデータが観測される確率が９０％以上になるまで、
振幅増幅を複数回おこなってください。

どのようなデータが探索されているでしょうか？
適切な振幅増幅の回数はなんでしょうか？

**解答**

量子回路の作成



シミュレーション